In [ ]:
# %matplotlib ipympl

from matplotlib.pyplot import *
font = {'weight' : 'normal',
        'size'   : 16}
matplotlib.rc('font', **font)

from numpy import *
import numpy as np
from matplotlib.pyplot import *
import math as math
from qutip import *
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

### Helper functions

In [ ]:
%matplotlib inline
from matplotlib.pyplot import *
font = {'weight' : 'normal',
        'size'   : 16}
matplotlib.rc('font', **font)


def plot2D(x, y, data):
    
    dx = x[1]-x[0]
    dy = y[1]-y[0]
    x2 = append(x, x[-1]+dx) - dx/2.0
    y2 = append(y, y[-1]+dy) - dy/2.0

    x2mesh,y2mesh = np.meshgrid(x2, y2)
    pcolormesh(x2mesh,y2mesh,data,cmap= 'Blues', vmin = 0, vmax =1)
    colorbar()
    # axis([x2.min(),x2.max(),y2.min(),y2.max()])
    # #xticks(x)
#     yticks(y)

def tri_area(p1,p2,p3):
    # p1,p2,p3 will be a complex number
    # example p1 = x+1j*y
    
    v1 = p2-p1
    v2 = p3-p1
    s = abs(real(v1)*imag(v2) - real(v2)*imag(v1))
    s = s/2
    
    return s

def gef_map(pg,pe,pf,p):
    s_total = tri_area(pg,pe,pf)
    s_g = tri_area(pe,pf,p)/s_total
    s_e = tri_area(pg,pf,p)/s_total
    s_f = tri_area(pg,pe,p)/s_total
    p_list = [s_g,s_e,s_f]
    return(p_list)

# function to randomize delta
def randomize_delta(delta, val, mag = 1):
    new_delta = delta
    for k in range(val): # randomly set a value in the array k times (increasingly more random as we go)
        random_index_i = random.randint(0, Nsite)
        random_index_j = random.randint(0, Msite)
        
        # if random.choice([True, False]):
        #     new_delta[random_index_i][random_index_j] = random.rand() * -mag
        # else:
        #     new_delta[random_index_i][random_index_j] = random.rand() * mag

        new_delta[random_index_i][random_index_j] = random.rand() * mag
        
    return new_delta

# Defining a N x 1 lattice

## Defining a Quantum Walk Function

In [ ]:
def quantum_walk_1D(Nsite, Msite, Ncutoff, photon_cutoff, n0, tlist, J_v, J_h, delta, U):
    # Defines an N x M lattice
    # Ncutoff levels in each site and photon_cutoff total photons allowed in the lattice
    # n0 is the inital state and tlist is the evolution
    # J_v is the vertical coupling strength and J_h is the horizontal coupling strength
    # delta is an energy detuning between sites (an array with an index for each site [Nsite x Msite]) <- this is what we want to disorder
    # U is the two particle interaction term <- set to 0 at first

    # Operators
    a_list = enr_destroy(dims=Ncutoff, excitations=photon_cutoff)

    n_list = []

    for a in a_list:
        n_list.append(a.dag() * a)

    expect_list = n_list

    # Initial State
    psi0 = enr_fock(dims=Ncutoff, excitations=photon_cutoff, state=n0)

    # Hamiltonian
    H = 0

    # tunneling across vertical
    for i in range(Nsite-1): # travel vertially
        for j in range(Msite): # travel horizontally
            H += -J_v*2*np.pi * (a_list[j + i*(Msite)].dag() * a_list[j + i*(Msite) + Msite]+ a_list[j + i*(Msite) + Msite].dag() * a_list[j + i*(Msite)])
            # print('('+str(j + i*(Msite)) + ',' + str(j + i*(Msite) + Msite) + ')')
    # detuning across sites
    for i in range(Nsite):
        for j in range(Msite):
            H += -delta[i][j]*2*np.pi * (a_list[j + i*(Msite)].dag() * a_list[j + i*(Msite)])

    ## Nearest Neighbor Interaction Term ##
    # across vertical
    for i in range(Nsite-1): # travel vertially
        for j in range(Msite): # travel horizontally
            H += -U[i][j]*2*np.pi * (a_list[j + i*(Msite)].dag()*a_list[j + i*(Msite)]) * (a_list[j + i*(Msite) + Msite].dag()*a_list[j + i*(Msite) + Msite])
            # print('('+str(j + i*(Nsite)) + ',' + str(j + i*(Nsite) + Msite) + ')')

    ##################
    #### master eqn.
    ##################
    medata = mesolve(H, psi0, tlist, c_ops = None, e_ops = expect_list, options = Options(store_states=True))

    return medata

### Now a side by side comparison

In [ ]:
Nsite = 20
Msite = 1
tot_site = Nsite * Msite
Ncutoff = [4]*tot_site
photon_cutoff = 3
n0 = [0]*tot_site

# random initial state
for i in range(photon_cutoff):
    random_index = random.randint(0, tot_site)
    n0[random_index] = 1
# n0[1] = 1
# n0[4] = 1

# Hamiltonian parameter: in MHz
# parameters: U for on-site interaction term and J for tunneling term
J_v = 100 # vertical tunneling strength (J / 2pi)
J_h = 100 # horizontal tunneling strength (J / 2pi)
delta_mag = J_v*10# Magnitude of delta for both
delta_d = randomize_delta(np.zeros((Nsite,Msite)), val=tot_site, mag=delta_mag) # disordered energy of the sites (epsilon / 2pi)
delta = np.full((Nsite,Msite), np.average(delta_d)) # fixed energy of the sites (epsilon / 2pi)
U = np.full((Nsite,Msite), 100) # Nearest-Neighbor interaction term (U / 2pi)
t_max = J_v * 0.5 * 10**-3
tlist = linspace(0, t_max, 200)


# Operators
a_list = enr_destroy(dims=Ncutoff, excitations=photon_cutoff)

n_list = []
for a in a_list:
    n_list.append(a.dag() * a)

# mesolve
medata = quantum_walk_1D(Nsite, Msite, Ncutoff, photon_cutoff, n0, tlist, J_v, J_h, delta, U)
medata_disorder = quantum_walk_1D(Nsite, Msite, Ncutoff, photon_cutoff, n0, tlist, J_v, J_h, delta_d, U)

In [ ]:
# Calculating the average photon number <n> in each site

# Reshape the data to fit the lattice dimensions (time, Nsite, Msite)
population_data = np.array(medata.expect).T  # Transpose to align with time x sites
population_data = population_data.reshape(-1, Nsite, Msite)

avg_n = np.zeros((Nsite, Msite))
for N in range(Nsite):
    for M in range(Msite):
        for t in range(len(tlist)):
            avg_n[N][M]+=population_data[t][N][M]
        avg_n[N][M] = avg_n[N][M] / len(tlist)


population_data_d = np.array(medata_disorder.expect).T  # Transpose to align with time x sites
population_data_d = population_data_d.reshape(-1, Nsite, Msite)

avg_n_d = np.zeros((Nsite, Msite))
for N in range(Nsite):
    for M in range(Msite):
        for t in range(len(tlist)):
            avg_n_d[N][M]+=population_data_d[t][N][M]
        avg_n_d[N][M] = avg_n_d[N][M] / len(tlist)



In [ ]:
%matplotlib qt

vmax = 1

fig,[ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
heatmap1 = ax1.imshow(avg_n, cmap="Blues", interpolation="nearest", aspect="auto", origin="lower", vmax=vmax)
heatmap2 = ax2.imshow(avg_n_d, cmap="Blues", interpolation="nearest", aspect="auto", origin="lower", vmax=vmax)

# Configure colorbar
cbar1 = plt.colorbar(heatmap1, ax=ax1)
cbar1.set_label(r"$\bar{\langle n \rangle}$")

# Titles and labels
ax1.set_title("Average Particle Population")
ax1.set_xlabel("M-site (Horizontal)")
ax1.set_ylabel("N-site (Vertical)")
ax1.set_yticks(range(Nsite))
ax1.set_xticks(range(Msite))

# Configure colorbar
cbar2 = plt.colorbar(heatmap2, ax=ax2)
cbar2.set_label(r"$\bar{\langle n \rangle}$")

# Titles and labels
ax2.set_title("Average Particle Population")
ax2.set_xlabel("M-site (Horizontal)")
ax2.set_ylabel("N-site (Vertical)")
ax2.set_yticks(range(Nsite))
ax2.set_xticks(range(Msite))

plt.tight_layout()
plt.show()

In [ ]:
fig.savefig('1Davg.png')

Plotting Disorder Across each lattice

In [ ]:
fig, [ax0a, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# For the uniform plot
heatmap_0a = ax0a.imshow(delta, cmap="Blues", interpolation="nearest", aspect="auto", origin="lower")

# Configure colorbar
cbar0a = plt.colorbar(heatmap_0a, ax=ax0a)
cbar0a.set_label(r"$\epsilon_i$")

# Titles and labels
ax0a.set_title(r"$\epsilon$ Value across sites")
ax0a.set_xlabel("M-site (Horizontal)")
ax0a.set_ylabel("N-site (Vertical)")
ax0a.set_yticks(range(Nsite))
ax0a.set_xticks(range(Msite))

# For the disorder plot
heatmap_2 = ax2.imshow(delta_d, cmap="Blues", interpolation="nearest", aspect="auto", origin="lower")

# Configure colorbar
cbar2 = plt.colorbar(heatmap_2, ax=ax2)
cbar2.set_label(r"$\epsilon_i$")

# Titles and labels
ax2.set_title(r"$\epsilon$ Value across sites")
ax2.set_xlabel("M-site (Horizontal)")
ax2.set_ylabel("N-site (Vertical)")
ax2.set_yticks(range(Nsite))
ax2.set_xticks(range(Msite))

# Display the animation
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig('1Depsilon.png')

In [ ]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib

# Reshape the data to fit the lattice dimensions (time, Nsite, Msite)
population_data = np.array(medata.expect).T  # Transpose to align with time x sites
population_data = population_data.reshape(-1, Nsite, Msite)

population_data_d = np.array(medata_disorder.expect).T  # Transpose to align with time x sites
population_data_d = population_data_d.reshape(-1, Nsite, Msite)

# Debug: Check shape of population_data
# print("Shape of population_data:", population_data.shape)

# Set up the figure for the live plot
fig, [ax0, ax1] = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

heatmap_0 = ax0.imshow(population_data[0], cmap="Blues", interpolation="nearest", aspect="auto", origin="lower")

# Configure colorbar
cbar0 = plt.colorbar(heatmap_0, ax=ax0)
cbar0.set_label(r"$\langle n \rangle$")

# Titles and labels
ax0.set_title("Particle Population on NxM Lattice")
ax0.set_xlabel("M-site (Horizontal)")
ax0.set_ylabel("N-site (Vertical)")
ax0.set_yticks(range(Nsite))
ax0.set_xticks(range(Msite))

heatmap_1 = ax1.imshow(population_data_d[0], cmap="Blues", interpolation="nearest", aspect="auto", origin="lower")

# Configure colorbar
cbar1 = plt.colorbar(heatmap_1, ax=ax1)
cbar1.set_label(r"$\langle n \rangle$")

# Titles and labels
ax1.set_title("Particle Population on NxM Lattice")
ax1.set_xlabel("M-site (Horizontal)")
ax1.set_ylabel("N-site (Vertical)")
ax1.set_yticks(range(Nsite))
ax1.set_xticks(range(Msite))



# Function to update the heatmap
def update(frame):
    # print(f"Updating frame {frame}, time = {tlist[frame] * J_v / (2 * np.pi):.2f}")  # Debug: Track updates
    heatmap_0.set_data(population_data[frame])
    ax0.set_title(r"Time =" + str(round(tlist[frame] * J_v / (2 * np.pi), 4)) + r"($t J / 2 \pi$)")
    heatmap_1.set_data(population_data_d[frame])
    ax1.set_title(r"Time =" + str(round(tlist[frame] * J_v / (2 * np.pi), 4)) + r"($t J / 2 \pi$)")
    return heatmap_0, heatmap_1,

# Create animation
anim = FuncAnimation(fig, update, frames=len(tlist), interval=10, blit=False)

# Display the animation
plt.tight_layout()
plt.show()


In [ ]:
anim.save('1Ddynamics.gif', writer='pillow', fps=60)


## Now with periodic boundary conditions

In [ ]:
def periodic_quantum_walk_1D(Nsite, Msite, Ncutoff, photon_cutoff, n0, tlist, J_v, J_h, delta, U):
    # Defines an N x M lattice
    # Ncutoff levels in each site and photon_cutoff total photons allowed in the lattice
    # n0 is the inital state and tlist is the evolution
    # J_v is the vertical coupling strength and J_h is the horizontal coupling strength
    # delta is an energy detuning between sites (an array with an index for each site [Nsite x Msite]) <- this is what we want to disorder
    # U is the two particle interaction term <- set to 0 at first

    # Operators
    a_list = enr_destroy(dims=Ncutoff, excitations=photon_cutoff)

    n_list = []

    for a in a_list:
        n_list.append(a.dag() * a)

    expect_list = n_list

    # Initial State
    psi0 = enr_fock(dims=Ncutoff, excitations=photon_cutoff, state=n0)

    # Hamiltonian
    H = 0

    # tunneling across vertical
    for i in range(Nsite): # travel vertially
        for j in range(Msite): # travel horizontally
            if i < Nsite-1:
                H += -J_v*2*np.pi * (a_list[j + i*(Msite)].dag() * a_list[j + i*(Msite) + Msite]+ a_list[j + i*(Msite) + Msite].dag() * a_list[j + i*(Msite)])
            else:
                H += -J_v*2*np.pi * (a_list[j + i*(Msite)].dag() * a_list[0]+ a_list[0].dag() * a_list[j + i*(Msite)])

            # print('('+str(j + i*(Msite)) + ',' + str(j + i*(Msite) + Msite) + ')')
    # detuning across sites
    for i in range(Nsite):
        for j in range(Msite):
            H += -delta[i][j]*2*np.pi * (a_list[j + i*(Msite)].dag() * a_list[j + i*(Msite)])

    ## Nearest Neighbor Interaction Term ##
    # across vertical
    for i in range(Nsite-1): # travel vertially
        for j in range(Msite): # travel horizontally
            if i < Nsite-1:
                H += -U[i][j]*2*np.pi * (a_list[j + i*(Msite)].dag()*a_list[j + i*(Msite)]) * (a_list[j + i*(Msite) + Msite].dag()*a_list[j + i*(Msite) + Msite])
            else:
                H += -U[i][j]*2*np.pi * (a_list[j + i*(Msite)].dag()*a_list[j + i*(Msite)]) * (a_list[0].dag()*a_list[0])
            # print('('+str(j + i*(Nsite)) + ',' + str(j + i*(Nsite) + Msite) + ')')

    ##################
    #### master eqn.
    ##################
    medata = mesolve(H, psi0, tlist, c_ops = None, e_ops = expect_list, options = Options(store_states=True))

    return medata

### Now a side by side comparison

In [ ]:
# mesolve
periodic_medata = periodic_quantum_walk_1D(Nsite, Msite, Ncutoff, photon_cutoff, n0, tlist, J_v, J_h, delta, U)
periodic_medata_disorder = periodic_quantum_walk_1D(Nsite, Msite, Ncutoff, photon_cutoff, n0, tlist, J_v, J_h, delta_d, U)

In [ ]:
# Calculating the average photon number <n> in each site

# Reshape the data to fit the lattice dimensions (time, Nsite, Msite)
population_data = np.array(periodic_medata.expect).T  # Transpose to align with time x sites
population_data = population_data.reshape(-1, Nsite, Msite)

avg_n = np.zeros((Nsite, Msite))
for N in range(Nsite):
    for M in range(Msite):
        for t in range(len(tlist)):
            avg_n[N][M]+=population_data[t][N][M]
        avg_n[N][M] = avg_n[N][M] / len(tlist)


population_data_d = np.array(periodic_medata_disorder.expect).T  # Transpose to align with time x sites
population_data_d = population_data_d.reshape(-1, Nsite, Msite)

avg_n_d = np.zeros((Nsite, Msite))
for N in range(Nsite):
    for M in range(Msite):
        for t in range(len(tlist)):
            avg_n_d[N][M]+=population_data_d[t][N][M]
        avg_n_d[N][M] = avg_n_d[N][M] / len(tlist)



In [ ]:
%matplotlib qt

vmax = 1

fig,[ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
heatmap1 = ax1.imshow(avg_n, cmap="Blues", interpolation="nearest", aspect="auto", origin="lower", vmax=vmax)
heatmap2 = ax2.imshow(avg_n_d, cmap="Blues", interpolation="nearest", aspect="auto", origin="lower", vmax=vmax)

# Configure colorbar
cbar1 = plt.colorbar(heatmap1, ax=ax1)
cbar1.set_label(r"$\bar{\langle n \rangle}$")

# Titles and labels
ax1.set_title("Average Particle Population")
ax1.set_xlabel("M-site (Horizontal)")
ax1.set_ylabel("N-site (Vertical)")
ax1.set_yticks(range(Nsite))
ax1.set_xticks(range(Msite))

# Configure colorbar
cbar2 = plt.colorbar(heatmap2, ax=ax2)
cbar2.set_label(r"$\bar{\langle n \rangle}$")

# Titles and labels
ax2.set_title("Average Particle Population")
ax2.set_xlabel("M-site (Horizontal)")
ax2.set_ylabel("N-site (Vertical)")
ax2.set_yticks(range(Nsite))
ax2.set_xticks(range(Msite))

plt.tight_layout()
plt.show()

In [ ]:
fig.savefig('1D_periodic_avg.png')

In [ ]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib

# Reshape the data to fit the lattice dimensions (time, Nsite, Msite)
population_data = np.array(periodic_medata.expect).T  # Transpose to align with time x sites
population_data = population_data.reshape(-1, Nsite, Msite)

population_data_d = np.array(periodic_medata_disorder.expect).T  # Transpose to align with time x sites
population_data_d = population_data_d.reshape(-1, Nsite, Msite)

# Debug: Check shape of population_data
# print("Shape of population_data:", population_data.shape)

# Set up the figure for the live plot
fig, [ax0, ax1] = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

heatmap_0 = ax0.imshow(population_data[0], cmap="Blues", interpolation="nearest", aspect="auto", origin="lower")

# Configure colorbar
cbar0 = plt.colorbar(heatmap_0, ax=ax0)
cbar0.set_label(r"$\langle n \rangle$")

# Titles and labels
ax0.set_title("Particle Population on NxM Lattice")
ax0.set_xlabel("M-site (Horizontal)")
ax0.set_ylabel("N-site (Vertical)")
ax0.set_yticks(range(Nsite))
ax0.set_xticks(range(Msite))

heatmap_1 = ax1.imshow(population_data_d[0], cmap="Blues", interpolation="nearest", aspect="auto", origin="lower")

# Configure colorbar
cbar1 = plt.colorbar(heatmap_1, ax=ax1)
cbar1.set_label(r"$\langle n \rangle$")

# Titles and labels
ax1.set_title("Particle Population on NxM Lattice")
ax1.set_xlabel("M-site (Horizontal)")
ax1.set_ylabel("N-site (Vertical)")
ax1.set_yticks(range(Nsite))
ax1.set_xticks(range(Msite))

# Function to update the heatmap
def update(frame):
    # print(f"Updating frame {frame}, time = {tlist[frame] * J_v / (2 * np.pi):.2f}")  # Debug: Track updates
    heatmap_0.set_data(population_data[frame])
    ax0.set_title(r"Time =" + str(round(tlist[frame] * J_v / (2 * np.pi), 4)) + r"($t J / 2 \pi$)")
    heatmap_1.set_data(population_data_d[frame])
    ax1.set_title(r"Time =" + str(round(tlist[frame] * J_v / (2 * np.pi), 4)) + r"($t J / 2 \pi$)")
    return heatmap_0, heatmap_1,

# Create animation
anim = FuncAnimation(fig, update, frames=len(tlist), interval=10, blit=False)

# Display the animation
plt.tight_layout()
plt.show()


In [ ]:
anim.save('1D_periodic_dynamics.gif', writer='pillow', fps=60)